# 1.1 - 1.3

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os

In [2]:
# Create path

path = r'G:\My Drive\CareerFoundry\Python Projects\2023-10 Instacart Basket Analysis'

In [3]:
# Imports customers data set
customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

# 1.4 Wrangle the data

In [5]:
# Get basic idea of data

customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


In [6]:
customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [8]:
# list of dtypes to include

include = ['object', 'float', 'int']

In [9]:
customers.describe(include=include)

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
count,206209.000000,194950,206209,206209,206209,206209.000000,206209,206209.000000,206209,206209.000000
unique,NaN,207,1000,2,51,NaN,1187,NaN,4,NaN
top,NaN,Marilyn,Hamilton,Male,Florida,NaN,9/17/2018,NaN,married,NaN
freq,NaN,2213,252,104067,4044,NaN,213,NaN,144906,NaN
mean,103105.000000,NaN,NaN,NaN,NaN,49.501646,NaN,1.499823,NaN,94632.852548
std,59527.555167,NaN,NaN,NaN,NaN,18.480962,NaN,1.118433,NaN,42473.786988
min,1.000000,NaN,NaN,NaN,NaN,18.000000,NaN,0.000000,NaN,25903.000000
25%,51553.000000,NaN,NaN,NaN,NaN,33.000000,NaN,0.000000,NaN,59874.000000
50%,103105.000000,NaN,NaN,NaN,NaN,49.000000,NaN,1.000000,NaN,93547.000000
75%,154657.000000,NaN,NaN,NaN,NaN,66.000000,NaN,3.000000,NaN,124244.000000


In [10]:
# Drop date_joined col. - unnecessary to analysis parameters

ic_customers = customers.drop('date_joined', axis=1)

In [11]:
ic_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1,married,40374


In [12]:
# Rename First Name, Surnam, Gender, State, Age and n_dependants for clarity and consistency

ic_customers.rename(columns = {'First Name' : 'first_name', 'Surnam' : 'last_name', 'Gender' : 'gender', 'STATE' : 'state', 'Age' : 'age', 'n_dependants' : 'dependants'}, inplace = True)

In [13]:
# Check result

ic_customers.columns

Index(['user_id', 'first_name', 'last_name', 'gender', 'state', 'age',
       'dependants', 'fam_status', 'income'],
      dtype='object')

# 1.5 Fundamental data quality and consistency checks

In [14]:
# Dtypes - change user_id to object ('str')

ic_customers[['user_id']] = ic_customers[['user_id']].astype(str)

In [15]:
ic_customers.dtypes

user_id       object
first_name    object
last_name     object
gender        object
state         object
age            int64
dependants     int64
fam_status    object
income         int64
dtype: object

In [16]:
# Check for mixed-type columns

for col in ic_customers.columns.tolist():
  weird = (ic_customers[[col]].applymap(type) != ic_customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (ic_customers[weird]) > 0:
    print (col)

first_name


In [19]:
# Instead of fixing the mixed-type first_name column I will remove it and the last_name column. Neither is needed for this ad-hoc analysis.

ic_customers = ic_customers.drop(['first_name', 'last_name'], axis=1)

In [20]:
ic_customers.columns

Index(['user_id', 'gender', 'state', 'age', 'dependants', 'fam_status',
       'income'],
      dtype='object')

In [21]:
# Check for/confirm missing values (info() showed only first_name had missing values)

ic_customers.isnull().sum()

user_id       0
gender        0
state         0
age           0
dependants    0
fam_status    0
income        0
dtype: int64

In [22]:
# Check for duplicates of entire df

ic_customers_dups = ic_customers[ic_customers.duplicated()]

In [23]:
ic_customers_dups

,user_id,gender,state,age,dependants,fam_status,income


In [24]:
# NO DUPS

In [26]:
ic_customers.head()

,user_id,gender,state,age,dependants,fam_status,income
0,26711,Female,Missouri,48,3,married,165665
1,33890,Female,New Mexico,36,0,single,59285
2,65803,Male,Idaho,35,2,married,99568
3,125935,Female,Iowa,40,0,single,42049
4,130797,Female,Maryland,26,1,married,40374


In [25]:
# Confirm unique values in state, fam_status and income are logical

In [27]:
ic_customers.describe(include=include)

,user_id,gender,state,age,dependants,fam_status,income
count,206209,206209,206209,206209.000000,206209.000000,206209,206209.000000
unique,206209,2,51,NaN,NaN,4,NaN
top,26711,Male,Florida,NaN,NaN,married,NaN
freq,1,104067,4044,NaN,NaN,144906,NaN
mean,NaN,NaN,NaN,49.501646,1.499823,NaN,94632.852548
std,NaN,NaN,NaN,18.480962,1.118433,NaN,42473.786988
min,NaN,NaN,NaN,18.000000,0.000000,NaN,25903.000000
25%,NaN,NaN,NaN,33.000000,0.000000,NaN,59874.000000
50%,NaN,NaN,NaN,49.000000,1.000000,NaN,93547.000000
75%,NaN,NaN,NaN,66.000000,3.000000,NaN,124244.000000


In [28]:
print(ic_customers['state'].unique())

['Missouri' 'New Mexico' 'Idaho' 'Iowa' 'Maryland' 'Kentucky' 'Montana'
 'South Carolina' 'Texas' 'Virginia' 'Nevada' 'Nebraska' 'Georgia'
 'Wyoming' 'Colorado' 'North Dakota' 'Wisconsin' 'Alaska' 'Vermont'
 'Arkansas' 'Maine' 'North Carolina' 'West Virginia' 'Indiana' 'Oregon'
 'Florida' 'California' 'Pennsylvania' 'Ohio' 'Connecticut' 'Arizona'
 'Louisiana' 'Washington' 'New York' 'Mississippi' 'Oklahoma' 'Utah'
 'New Hampshire' 'Hawaii' 'District of Columbia' 'Alabama' 'Massachusetts'
 'Rhode Island' 'Michigan' 'New Jersey' 'Kansas' 'South Dakota'
 'Minnesota' 'Illinois' 'Tennessee' 'Delaware']


In [29]:
print(ic_customers['fam_status'].unique())

['married' 'single' 'living with parents and siblings' 'divorced/widowed']


In [32]:
# Max income seems high, check to see how many values are greater than the Q3 threshold

(ic_customers['income'] > 124244).sum()

51551

In [ ]:
# Too many instances to remove - confer with data source owners

# 1.6 Combine your customer data with the rest of your prepared Instacart data

In [34]:
# Import ords_prods_merge_grp_agg to combine with ic_customers

ords_prods = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_grp_agg.pkl'))

In [35]:
ords_prods.head()

,order_id,user_id,number_of_orders,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_item_price,spending_flag,prior_order_median,order_freq_flag
0,2539329,1,1,2,8,0.0,196,1,0,Soda,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.0,Regular customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.0,Regular customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.0,Regular customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Mid-range product,Least busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.0,Regular customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Mid-range product,Least busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.0,Regular customer


In [36]:
ords_prods.columns

Index(['order_id', 'user_id', 'number_of_orders', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices', '_merge', 'price_range_loc', 'busiest_day',
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'avg_item_price', 'spending_flag', 'prior_order_median',
       'order_freq_flag'],
      dtype='object')

In [38]:
# Remove unneeded columns

ords_prods_merge = ords_prods.drop(['_merge', 'busiest_day', 'max_order', 'avg_item_price', 'prior_order_median'], axis=1)

In [39]:
ords_prods_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32404859 entries, 0 to 32404858
Data columns (total 19 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   number_of_orders        int64  
 3   orders_day_of_week      int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
 6   product_id              int64  
 7   add_to_cart_order       int64  
 8   reordered               int64  
 9   product_name            object 
 10  aisle_id                int64  
 11  department_id           int64  
 12  prices                  float64
 13  price_range_loc         object 
 14  busiest_days            object 
 15  busiest_period_of_day   object 
 16  loyalty_flag            object 
 17  spending_flag           object 
 18  order_freq_flag         object 
dtypes: float64(2), int64(10), object(7)
memory usage: 4.6+ GB


In [40]:
del ords_prods

In [41]:
# Change ords_prods_merge 'user_id' dtype to object to match 'user_id' in ic_customers

ords_prods_merge[['user_id']] = ords_prods_merge[['user_id']].astype(str)

In [43]:
ords_prods_merge['user_id'].dtype

dtype('O')

In [ ]:
ords_prods_merge.describe(include=include)

In [45]:
# Use Inner Join to merge but first test merge without overwriting

pd.merge(ic_customers,ords_prods_merge, on = ['user_id'], indicator = True)

,user_id,gender,state,age,dependants,fam_status,income,order_id,number_of_orders,orders_day_of_week,...,aisle_id,department_id,prices,price_range_loc,busiest_days,busiest_period_of_day,loyalty_flag,spending_flag,order_freq_flag,_merge
0,26711,Female,Missouri,48,3,married,165665,2543867,5,1,...,77,7,9.0,Mid-range product,Busiest days,Most orders,New customer,Low spender,Regular customer,both
1,26711,Female,Missouri,48,3,married,165665,1285508,7,5,...,77,7,9.0,Mid-range product,Regularly busy,Most orders,New customer,Low spender,Regular customer,both
2,26711,Female,Missouri,48,3,married,165665,2578584,8,1,...,77,7,9.0,Mid-range product,Busiest days,Most orders,New customer,Low spender,Regular customer,both
3,26711,Female,Missouri,48,3,married,165665,423547,2,2,...,120,16,12.6,Mid-range product,Regularly busy,Most orders,New customer,Low spender,Regular customer,both
4,26711,Female,Missouri,48,3,married,165665,2524893,3,3,...,120,16,12.6,Mid-range product,Least busy days,Most orders,New customer,Low spender,Regular customer,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32404854,80148,Female,New York,55,1,married,57095,2859858,3,6,...,3,19,4.7,Low-range product,Regularly busy,Most orders,New customer,Low spender,Regular customer,both
32404855,80148,Female,New York,55,1,married,57095,2859858,3,6,...,3,19,4.0,Low-range product,Regularly busy,Most orders,New customer,Low spender,Regular customer,both
32404856,80148,Female,New York,55,1,married,57095,3209855,4,6,...,26,7,4.7,Low-range product,Regularly busy,Average orders,New customer,Low spender,Regular customer,both
32404857,80148,Female,New York,55,1,married,57095,2859858,3,6,...,3,19,1.8,Low-range product,Regularly busy,Most orders,New customer,Low spender,Regular customer,both


In [46]:
# Merge

ords_prods_cust_merged = ords_prods_merge.merge(ic_customers, on = 'user_id')

In [47]:
ords_prods_cust_merged.head()

,order_id,user_id,number_of_orders,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,busiest_period_of_day,loyalty_flag,spending_flag,order_freq_flag,gender,state,age,dependants,fam_status,income
0,2539329,1,1,2,8,0.0,196,1,0,Soda,...,Average orders,New customer,Low spender,Regular customer,Female,Alabama,31,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Average orders,New customer,Low spender,Regular customer,Female,Alabama,31,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Most orders,New customer,Low spender,Regular customer,Female,Alabama,31,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Average orders,New customer,Low spender,Regular customer,Female,Alabama,31,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Most orders,New customer,Low spender,Regular customer,Female,Alabama,31,3,married,40423


In [48]:
ords_prods_cust_merged.shape

(32404859, 25)

In [49]:
# Export new df as pkl

ords_prods_cust_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_cust_merged.pkl'))